<a href="https://colab.research.google.com/github/Vcarmelli/Peptide-Generation/blob/main/PROJECT_Inhibition_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch transformers sentencepiece h5py --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00


In [ ]:
#@title Load Model
from transformers import T5EncoderModel, T5Tokenizer
import numpy as np
import torch
import h5py

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # check whether GPU is available
print("Using {}".format(device))

model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
model = model.to(device) # move model to GPU
model = model.eval() # set model to evaluation model
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)

Using cuda:0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#@title Pre-process dataset
import pandas as pd

csv_folder = "/content/drive/MyDrive/csv/Final Datasets"
dataset = pd.read_csv(f'{csv_folder}/peptides_with_labels.csv')
generated = pd.read_csv(f'{csv_folder}/generated_peptides.csv')

def preprocess(row):
    target = row["Target"]
    sequence_id = row["ID"]
    sequence = row["Sequence"]
    inhibition = row["IC50"] if pd.notna(row["IC50"]) else 0
    return pd.Series([target, sequence_id, sequence, inhibition])

dataset[['serotype', 'sequence_id', 'peptide', 'inhibition']] = dataset.apply(preprocess, axis=1)
dataset = dataset[['serotype', 'sequence_id', 'peptide', 'inhibition']]
dataset.head()

,serotype,sequence_id,peptide,inhibition
0,1,DRAVPe01639,GQGKAHNGRLITANP,33.0
1,2,DRAVPe01639,GQGKAHNGRLITANP,10.0
2,3,DRAVPe01639,GQGKAHNGRLITANP,20.0
3,4,DRAVPe01639,GQGKAHNGRLITANP,20.0
4,1,DRAVPe01638,DRGWGNGCGLFG,10.0


In [ ]:
generated.head()  # generated peptide saved from other notebook

,serotype,sequence_id,peptide
0,1,DENV1SEQ01,RRRKTCG
1,1,DENV1SEQ02,ADLPEGW
2,1,DENV1SEQ03,EGRLKF
3,1,DENV1SEQ04,RRHFCW
4,1,DENV1SEQ05,RGKRPAC


In [ ]:
#@title Set up working directories

!mkdir protT5
!mkdir protT5/protT5_checkpoint
!mkdir protT5/output

dataset_embed_path = "./protT5/output/dataset_embeddings.h5"
peptide_embed_path = "./protT5/output/peptide_embeddings.h5"

mkdir: cannot create directory ‘protT5’: File exists
mkdir: cannot create directory ‘protT5/protT5_checkpoint’: File exists
mkdir: cannot create directory ‘protT5/output’: File exists


In [ ]:
#@title Load functions for embeddings

import numpy as np

def get_embeddings( model, tokenizer, seqs, per_protein=True,
                    max_residues=900, max_seq_len=300, max_batch=100 ):

    results = {"protein_embs" : dict()}
    seq_dict = sorted(
        ((k, v) for k, v in seqs.items() if isinstance(v, str)),
        key=lambda kv: len(kv[1]),
        reverse=True
    )
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)

            try:
                with torch.no_grad():
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if per_protein:
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    print('Total number of per-protein embeddings: {}'.format( len(results["protein_embs"] )))
    return results


# Write embeddings to disk
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "w") as hf:
        for sequence_id, embedding in emb_dict.items():
            hf.create_dataset(sequence_id, data=embedding)
    return None

# Retrieve embeddings from the disk
def read_embeddings(file_path):
  embeddings = {}
  with h5py.File(file_path, "r") as f:
    for key in f.keys():
      embeddings[key] = f[key][:]
  return embeddings

def add_embeddings_to_dataframe(df, embedding_dict):
    def get_embedding(row):
        return embedding_dict.get(row["sequence_id"], np.nan)

    df["embedding"] = df.apply(get_embedding, axis=1)
    return df


def get_sequence_dictionary(df):
    seq_dict = {
        str(row['sequence_id']): row['peptide']
        for _, row in df.iterrows()
    }
    return seq_dict

In [ ]:
#@title Compute embeddings for dataset peptides

seq_dict = get_sequence_dictionary(dataset)
results = get_embeddings(model, tokenizer, seq_dict)
save_embeddings(results["protein_embs"], dataset_embed_path)
dataset_embeddings = read_embeddings(dataset_embed_path)
dataset_embed_df = add_embeddings_to_dataframe(dataset, dataset_embeddings)
dataset_embed_df.head()

TypeError: 'XGBRegressor' object is not callable

In [ ]:
#@title Compute embeddings for generated peptides

seq_dict = get_sequence_dictionary(generated)
results = get_embeddings(model, tokenizer, seq_dict)
save_embeddings(results["protein_embs"], peptide_embed_path)
peptide_embeddings = read_embeddings(peptide_embed_path)
peptide_embed_df = add_embeddings_to_dataframe(generated, peptide_embeddings)
peptide_embed_df.head()

TypeError: 'XGBRegressor' object is not callable

In [ ]:
#@title Use XGBoost Regressor to Predict Inhibition

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def load_features(dataframe):
    le = LabelEncoder()
    embeds = np.vstack(dataframe["embedding"].values)
    serotype_encoded = le.fit_transform(dataframe["serotype"]) # transform numerical serotype as categorical values
    serotype_encoded = serotype_encoded.reshape(-1, 1)
    combined = np.hstack([embeds, serotype_encoded])
    return combined

X_combined = load_features(dataset_embed_df) # combine serotype and embedding as features for model training

y = dataset_embed_df["inhibition"].values  # target column to predict
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

#model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


X_new_combined = load_features(peptide_embed_df) # using the generated peptides data

y_new_pred = model.predict(X_new_combined)
peptide_embed_df["predicted_inhibition"] = np.abs(y_new_pred)
peptide_embed_df

,serotype,sequence_id,peptide,embedding,predicted_inhibition
0,1,DENV1SEQ01,RRRKTCG,"[0.1682114, -0.053856615, -0.124487355, -0.052...",218.06551
1,1,DENV1SEQ02,ADLPEGW,"[0.17050067, -0.020698216, -0.08562408, -0.060...",194.48030
2,1,DENV1SEQ03,EGRLKF,"[0.2331349, 0.027426898, -0.078136176, -0.1116...",340.22510
3,1,DENV1SEQ04,RRHFCW,"[0.20430134, 0.033243284, -0.15178554, -0.0862...",5.16820
4,1,DENV1SEQ05,RGKRPAC,"[0.19530068, -0.05945772, -0.14176694, -0.0972...",23.75881
5,1,DENV1SEQ06,AEGHW,"[0.15526296, -0.025776181, -0.2089316, -0.1412...",198.63780
6,1,DENV1SEQ07,PGKRKSC,"[0.13939357, -0.047078382, -0.12242581, -0.075...",17.14510
7,1,DENV1SEQ08,RRRRYWCGY,"[0.096577734, 0.05730789, -0.16303885, -0.1248...",1.42360
8,1,DENV1SEQ09,RGWFAKCTV,"[0.11112314, 0.10824275, -0.044924583, -0.1450...",39.47931
9,1,DENV1SEQ10,AGYHPEFDL,"[0.1493098, 0.016891228, -0.09184324, -0.11081...",71.50858


In [ ]:
#@title Save the predicted inhibition to a new file

peptide_embed_df.to_csv(f"{csv_folder}/generated_peptides_with_inhibition.csv", index=False)
print(f"Peptides saved as generated_peptides_with_inhibition.csv in {csv_folder}")

Peptides saved as generated_peptides_with_inhibition.csv in /content/drive/MyDrive/csv/Final Datasets


In [ ]:
top3_lowest_by_serotype = (
    peptide_embed_df
    .sort_values(by=["serotype", "predicted_inhibition"])
    .groupby("serotype")
    .head(3)
)

top3_lowest_by_serotype

,serotype,sequence_id,peptide,embedding,predicted_inhibition
7,1,DENV1SEQ08,RRRRYWCGY,"[0.096577734, 0.05730789, -0.16303885, -0.1248...",1.42360
3,1,DENV1SEQ04,RRHFCW,"[0.20430134, 0.033243284, -0.15178554, -0.0862...",5.16820
6,1,DENV1SEQ07,PGKRKSC,"[0.13939357, -0.047078382, -0.12242581, -0.075...",17.14510
18,2,DENV2SEQ09,RRRRGCSY,"[0.17260997, -0.003486432, -0.15887468, -0.034...",2.32591
16,2,DENV2SEQ07,GRRRWCY,"[0.1526489, 0.016872885, -0.22466743, -0.12945...",5.22920
10,2,DENV2SEQ01,RRHWCW,"[0.17510894, -0.014975963, -0.2411215, -0.0876...",10.56640
24,3,DENV3SEQ05,RRHWCWY,"[0.17614046, 0.020981159, -0.24420302, -0.0924...",16.91649
25,3,DENV3SEQ06,RGWLHKAY,"[0.18024242, 0.025007043, -0.09166317, -0.1100...",21.72380
28,3,DENV3SEQ09,LKRKH,"[0.11683402, -0.084480464, -0.033545744, -0.08...",40.15400
39,4,DENV4SEQ10,RRRRHWCCY,"[0.1167312, 0.043228377, -0.13554618, -0.04469...",1.53450
